In [22]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
df_ROSN = pd.read_csv('ROSN_after_EDA.csv')  
df_SBER = pd.read_csv('SBER_after_EDA.csv')  
df_LKOH = pd.read_csv('LKOH_after_EDA.csv')  
df_SI = pd.read_csv('SI_after_EDA.csv')  
df_BR = pd.read_csv('BR_after_EDA.csv')  
df_IMOEX = pd.read_csv('IMOEX_after_EDA.csv')  
df_STOCK_AGGREGATE = pd.read_csv('df_STOCK_AGGREGATE.csv')  

# Гипотезы

* **№ 1** | H0: Матожидание цены Роснефти за последнее полугодие (январь-апрель 2024) равняется цене после дивидендного гэпа в 2024 году.


* **№ 2** | H0: Курс рубля по отношению к доллару меняется вне зависимости от наличия или отсутствия налогового периода.


* **№ 3** | H0: Объявление о размерах дивидендов влияет на стоимость акций компаний сразу после объявления.

## Гипотеза №1

**Описание гипотезы:**

Дивидендный гэп – это резкое падение цены акций после выплаты дивидендов. Он возникает потому, что выплата дивидендов уменьшает стоимость компании на сумму выплаченных дивидендов. Если цена акций после гэпа быстро восстанавливается и матожидание цены в следующем полугодии остается на том же уровне, это свидетельствует о стабильности и устойчивости компании. Это может быть позитивным сигналом для инвесторов.

Изучение матожидания цены акций после дивидендного гэпа позволяет понять, насколько дивиденды влияют на долгосрочную стоимость акций. Это важно для инвесторов, заинтересованных в устойчивом доходе от дивидендов и стабильной стоимости акций.
Прогнозирование будущих цен на основе исторических данных:

Если матожидание цены в новом полугодии остается на уровне цены после дивидендного гэпа, это может использоваться для прогнозирования будущих цен. Инвесторы могут полагаться на эту информацию для принятия решений о покупке или продаже акций.
Оценка эффективности управления компанией:

Способность компании поддерживать стабильные цены акций после выплаты дивидендов может свидетельствовать о ее эффективном управлении и сильных фундаментальных показателях. Это может повышать доверие инвесторов и способствовать долгосрочной привлекательности акций компании.

**Проверка гипотезы**
Для начала соберем данные о цене Роснефти за последнее полугодие (январь-апрель 2024). В качестве цены будем брать среднюю цену за торговую сессию "WAPRICE"

In [39]:
df_ROSN['TRADEDATE'] = pd.to_datetime(df_ROSN['TRADEDATE'])

In [40]:
df_ROSN_2024 = df_ROSN[df_ROSN['TRADEDATE'].dt.year == 2024][['WAPRICE']]

In [41]:
df_ROSN_2024 = df_ROSN_2024.values.reshape(1,-1).tolist()[0]

Будем проверять гипотезу на основе t-теста. Для начала нам нужна цена после дивидендного гэпа в 2023 году. Дата выплаты девидентов по Роснефти является 11.01.2024 (11 января 2024 года). Значит необходимо брать цену сразу после девидентного гэпа  то есть 12 января.

In [64]:
mu = df_ROSN[(df_ROSN['TRADEDATE'].dt.year == 2024) & (df_ROSN['TRADEDATE'] == '2024-01-12')]['WAPRICE']

In [66]:
result = stats.ttest_1samp(df_ROSN_2024, mu)

Уровень значимости (или уровень доверия) 5% ($\alpha=0.05$) является общепринятым стандартом в статистике и науке. Это означает, что существует 5% вероятность (1 из 20) отклонить нулевую гипотезу, когда она на самом деле верна (ошибка первого рода).

In [68]:
a = 0.05

In [74]:
(result.pvalue > a)[0]

False

Отвержение гипотезы означает, что средняя цена акций Роснефти за последние полугодие (январь-апрель 2024) значимо отличается от цены после дивидендного гэпа в 2023 году. Это может указывать на значительные изменения в динамике цен акций, что требует дальнейшего анализа для понимания причин этих изменений.

## Гипотеза №2

**Описание гипотезы:**

Экспортеры получают доходы в иностранной валюте от продаж за рубежом.
В налоговый период компании-экспортеры обязаны выплачивать налоги в рублях.
Для этого экспортеры продают часть своих валютных резервов, чтобы купить рубли для уплаты налогов.

Увеличение предложения иностранной валюты на рынке приводит к укреплению рубля и снижению курса доллара.

В периоды уплаты налогов можно наблюдать увеличение объема продаж иностранной валюты.

Это сопровождается временным укреплением национальной валюты (рубля).

**Проверка гипотезы**

Для начала нам нужна информация касательно того в какие периоды проводятся налоговые периоды.

Влияние экспортеров на курс рубля
Экспортеры, как правило, продают свою валютную выручку за несколько дней до наступления сроков выплат налогов и других обязательных платежей. Обычно это происходит за 3-5 дней до дедлайна (25 числа каждого месяца происходят выплаты по НДПИ, НДС, акцизам; 28 числа – по налогу на прибыль.)
Соответственно, пик продаж валюты экспортерами приходится на период с 20 по 23 число каждого месяца. Это приводит к увеличению предложения долларов и евро на внутреннем валютном рынке России, что, в свою очередь, способствует укреплению курса рубля в эти дни.
Таким образом, можно ожидать, что основное укрепление российской национальной валюты по отношению к доллару и евро будет наблюдаться в период с 20 по 23 число каждого месяца, когда экспортеры активно продают свою валютную выручку.

In [183]:
data = df_SI[['TRADEDATE', 'WAPRICE']]

In [184]:
data

,TRADEDATE,WAPRICE
0,2021-01-04,74151.0
1,2021-01-05,75122.0
2,2021-01-06,74323.0
3,2021-01-08,74709.0
4,2021-01-11,75069.0
...,...,...
799,2024-03-26,95010.0
800,2024-03-27,94800.0
801,2024-03-28,94631.0
802,2024-03-29,94355.0


In [185]:
dates = data[['TRADEDATE']]

In [186]:
dates['Day'] = dates['TRADEDATE'].apply(lambda x: int(x.replace('-', ""))%100)

In [187]:
dates

,TRADEDATE,Day
0,2021-01-04,4
1,2021-01-05,5
2,2021-01-06,6
3,2021-01-08,8
4,2021-01-11,11
...,...,...
799,2024-03-26,26
800,2024-03-27,27
801,2024-03-28,28
802,2024-03-29,29


In [188]:
dates['Tax period']  = np.where(((dates['Day']>=20) & (dates['Day']<=23)), 1, 0)

In [189]:
dates['Tax period'].value_counts()

0    696
1    108
Name: Tax period, dtype: int64

1 Если день выпадает на налоговый период и 0, иначе.

In [190]:
data_course = df_SI[['TRADEDATE','TRENDCLSPR']]

In [191]:
data_course = data_course.copy()
data_course.loc[:, 'TRENDCLSPR_TAX'] = data_course['TRENDCLSPR'].apply(lambda x: 1 if x <= 0 else 0)

In [192]:
data_course.set_index('TRADEDATE', inplace=True)

In [193]:
dates.set_index('TRADEDATE', inplace=True)

In [194]:
data_course

,TRENDCLSPR,TRENDCLSPR_TAX
TRADEDATE,,
2021-01-04,-0.0097,1
2021-01-05,0.0043,0
2021-01-06,-0.0039,1
2021-01-08,0.0036,0
2021-01-11,0.0068,0
...,...,...
2024-03-26,0.0008,0
2024-03-27,-0.0080,1
2024-03-28,0.0038,0


In [195]:
df = pd.DataFrame(columns=['Dynamics', 'Tax period'])

In [218]:
df.dropna(inplace = True)

In [219]:
df['Dynamics'] = data_course['TRENDCLSPR_TAX']
df['Tax period'] = dates['Tax period']

In [220]:
df_2 = df.groupby('Tax period')['Dynamics'].value_counts().unstack()
df_2

Dynamics,0,1
Tax period,,
0,349,347
1,53,55


In [221]:
accuracy = (349+55)/(349+347+53+55)
accuracy

0.5024875621890548

In [223]:
a = stats.chi2_contingency(df_2, correction=False)
chi2=a[0]
p_value=a[1]

print('chi2-статистика =', chi2)
print('p-value =', p_value)

if p_value < 0.05:
    print("Нулевая гипотеза отвергается. Эффект налогового периода наблюдается")
else:
    print("Нулевая гипотеза не отвергается. Эффекта налогового периода не наблюдается")

chi2-статистика = 0.042784163473818644
p-value = 0.8361322147991934
Нулевая гипотеза не отвергается. Эффекта налогового периода не наблюдается


Разница между наблюдаемыми частотами для различных категорий (налоговый период и динамика курса) была слишком мала, чтобы быть статистически значимой. Даже если различия существуют, они могут быть слишком малы, чтобы обнаружить их с помощью используемых методов.
Ограниченность данных:

Возможно, наши данные не охватывают достаточно длительный период или не содержат достаточно наблюдений, чтобы адекватно выявить существование эффекта налогового периода на курс рубля. Более длинные временные ряды и дополнительные данные могли бы дать более точные результаты.
Влияние других факторов:

Курс рубля по отношению к доллару определяется множеством факторов, таких как мировые цены на нефть, монетарная политика, международные санкции, политические события и т.д. Влияние налогового периода могло быть подавлено или замаскировано этими другими, более сильными факторами.
Сезонные и случайные колебания:

Валютные курсы подвержены сезонным и случайным колебаниям, которые могут сильно варьироваться. Налоговый период мог не быть достаточно значимым фактором, чтобы проявиться на фоне этих колебаний.
Заключение

Наш анализ не предоставил достаточных доказательств в пользу того, что налоговый период оказывает значительное влияние на курс рубля по отношению к доллару. Это может быть связано с вышеупомянутыми причинами, и дальнейшие исследования с более обширными и подробными данными могли бы предоставить более ясную картину.

## Гипотеза №3

**Описание гипотезы**

Гипотеза о том, что объявление о размерах дивидендов влияет на стоимость акций компаний до момента выплаты имеет множество предпосылок. Объявление о выплате дивидендов несет в себе дополнительную информацию для фондового рынка, являясь сигналом для инвесторов. Это может привести к изменению стоимости акций компании еще до момента выплаты дивидендов. 
Кроме того, инвесторы могут начать покупать акции уже после выхода отчетности компании, ожидая объявления размера дивидендов. Это может привести к росту стоимости акций до момента выплаты дивидендов.Обычно после объявления дивидендов акции компании растут в цене, и этот рост может продолжаться вплоть до даты отсечения. Это происходит потому, что инвесторы ожидают получения дивидендов и готовы платить больше за акции, которые обеспечивают дополнительный доход.
Таким образом, объявление о размерах дивидендов может влиять на стоимость акций компаний до момента выплаты, так как инвесторы учитывают ожидаемый доход от дивидендов при принятии инвестиционных решений.

**Проверка гипотезы**

Для подтверждения гипотезы, мы намерены рассмотреть динамику акций в день объявления дивидендов, предполагается, что рынок будет позитивно реагировать ростом на положительные новости о предстоящей выплате дивидендов.

Для начала мы выписали даты объявления дивидендов за рассматриваемый в проекте период

In [227]:
d_dates = ['2021-03-04', '2021-08-24', '2022-05-30', '2022-11-14', '2023-05-30', '2023-11-15']

In [235]:
d_dates = pd.to_datetime(d_dates, format='%Y-%m-%d')

df_ROSN['TRADEDATE'] = pd.to_datetime(df_ROSN['TRADEDATE'], format='%Y-%m-%d')
df_ROSN_filtered = df_ROSN[df_ROSN['TRADEDATE'].isin(d_dates)]

df_ROSN_filtered = df_ROSN_filtered[['TRADEDATE', 'WAPRICE']]

In [240]:
df_ROSN_filtered['Delta'] = df_ROSN['TRENDCLSPR']

In [241]:
df_ROSN_filtered

,TRADEDATE,WAPRICE,Delta
42,2021-03-04,539.55,1.18
163,2021-08-24,543.35,0.03
354,2022-05-30,396.65,-4.08
472,2022-11-14,352.10,-1.71
608,2023-05-30,438.25,-1.35
728,2023-11-15,585.35,-1.07


In [239]:
df_ROSN['TRADEDATE'] = pd.to_datetime(df_ROSN['TRADEDATE'])
before_dates = df_ROSN[df_ROSN['TRADEDATE'].isin(['2021-03-03', '2021-08-23', '2022-05-27', '2022-11-11', '2023-05-29', '2023-11-14'])]
after_dates = df_ROSN[df_ROSN['TRADEDATE'].isin(['2021-03-04', '2021-08-24', '2022-05-30', '2022-11-14', '2023-05-30', '2023-11-15'])]

before_prices = before_dates['WAPRICE'].values
after_prices = after_dates['WAPRICE'].values

t_stat, p_value = stats.ttest_rel(before_prices, after_prices)

print(f"t-статистика: {t_stat}")
print(f"p-значение: {p_value}")

alpha = 0.05
if p_value > alpha:
    print("Нулевая гипотеза не отвергается: объявление о размерах дивидендов влияет на стоимость акций компаний.")
else:
    print("Нулевая гипотеза отвергается: нет статистически значимого влияния объявления о размерах дивидендов на стоимость акций компаний.")

t-статистика: 0.6603495603855019
p-значение: 0.538232604373106
Нулевая гипотеза не отвергается: объявление о размерах дивидендов влияет на стоимость акций компаний.


Гипотеза о положительной динамике акций в день объявления дивидендов подтверждается по вероятным нескольким причинам:

1. Реакция инвесторов на выплату дивидендов свидетельствует о том, что они предпочитают дивиденды нераспределенной прибыли

2. Объявление дивиденда служит позитивным сигналом рынку, что выражается в повышении цены акций в окне события.

3.  Выплата дивидендов в виде акций позволяет инвесторам приобрести новые акции без транзакционных издержек, а компании сохранить ликвидные ресурсы.

Таким образом, объявления о дивидендах содержат важную информацию для рынка, влияют на поведение инвесторов и цены акций, что подтверждает гипотезу о положительной динамике котировок в день объявления дивидендов